## Simple Transfer Learning example in VGG16

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.models import load_model
import keras.backend as K
import keras

from keras.applications.imagenet_utils import _obtain_input_shape
from keras import backend as K
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, \
    warnings

from keras.engine.topology import get_source_inputs
from keras.utils import get_file
from keras.utils import layer_utils

from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense



Using TensorFlow backend.


In [2]:
print(keras.__version__)

K.clear_session()


2.0.6


In [3]:
# dimensions of our images.
img_width, img_height = 229,229

#train_data_dir = './data/train/'
train_data_dir = '/home/samwit/Dropbox/ai_learning/Satay/satay_data/train/'
#validation_data_dir = './data/test/'
validation_data_dir = '/home/samwit/Dropbox/ai_learning/Satay/satay_data/valid/'
nb_train_samples = 2000
nb_validation_samples = 64
epochs = 10
batch_size = 16


In [4]:
#vgg input
vgg_inp=Input((229,229,3))

# build the VGG16 network
vgg = applications.VGG16(weights='imagenet', include_top=False, input_tensor=vgg_inp)
print('Model loaded.')

#vgg network
for l in vgg.layers: l.trainable=False


58892288/58889256 [==============================] - 19s    
Model loaded.
Model loaded.


In [5]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 229, 229, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 229, 229, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 229, 229, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 114, 114, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 114, 114, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 114, 114, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 57, 57, 128)       0         
__________

In [14]:
x = vgg.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

In [15]:
# this is the model we will train
model = Model(inputs=vgg.input, outputs=predictions)

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 229, 229, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 229, 229, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 229, 229, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 114, 114, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 114, 114, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 114, 114, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 57, 57, 128)       0         
__________

In [17]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


### Image Augmentation 

In [35]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)


# make the generators for getting the images
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 4134 images belonging to 2 classes.
Found 310 images belonging to 2 classes.


In [36]:
# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

/home/samwit/anaconda3/envs/coremltools/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/samwit/anaconda3/envs/coremltools/lib/python2.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=10, validation_data=<keras.pre..., steps_per_epoch=2000, validation_steps=64)`
  import sys


Epoch 1/10
2000/2000 [==============================] - 401s - loss: 0.2790 - acc: 0.8777 - val_loss: 0.1845 - val_acc: 0.9276
Epoch 2/10
2000/2000 [==============================] - 397s - loss: 0.1902 - acc: 0.9230 - val_loss: 0.1669 - val_acc: 0.9175
Epoch 3/10
2000/2000 [==============================] - 398s - loss: 0.1642 - acc: 0.9341 - val_loss: 0.1228 - val_acc: 0.9547
Epoch 4/10
2000/2000 [==============================] - 397s - loss: 0.1453 - acc: 0.9433 - val_loss: 0.1213 - val_acc: 0.9386
Epoch 5/10
2000/2000 [==============================] - 398s - loss: 0.1347 - acc: 0.9478 - val_loss: 0.0866 - val_acc: 0.9678
Epoch 6/10
2000/2000 [==============================] - 396s - loss: 0.1247 - acc: 0.9510 - val_loss: 0.1042 - val_acc: 0.9487
Epoch 7/10
2000/2000 [==============================] - 398s - loss: 0.1150 - acc: 0.9563 - val_loss: 0.0909 - val_acc: 0.9557
Epoch 8/10
2000/2000 [==============================] - 397s - loss: 0.1069 - acc: 0.9595 - val_loss: 0.1037 - 

In [20]:
model.save('satay_model_05.hd5')

In [ ]:
model = load_model('satay_model_05.hd5')

In [6]:
vgg2 = applications.VGG16(weights='imagenet', include_top=True, input_tensor=vgg_inp)

553467904/553467096 [==============================] - 170s   


In [7]:
vgg2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 229, 229, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 229, 229, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 229, 229, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 114, 114, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 114, 114, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 114, 114, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 57, 57, 128)       0         
__________

In [18]:
layer = vgg2.get_layer('fc2')

In [19]:
layer.activation

<function keras.activations.relu>